# 🎬 AI Video Clipper - Google Colab Edition
### Sistema 100% Open-Source de Criação de Clipes Virais

**Funcionalidades:**
- ✅ Transcrição local (Whisper/Vosk)
- ✅ Análise viral com Ollama
- ✅ Edição automática 9:16
- ✅ Legendas dinâmicas
- ✅ Upload automático para Google Drive
- ✅ **Interface Web Completa**

## 🔗 ETAPA 1: Conectar Google Drive e Carregar Projeto

In [ ]:
from google.colab import drive
import os
import sys

# 1. Montar Drive
print("📂 Montando Google Drive...")
drive.mount('/content/drive')

# 2. Definir caminhos
DRIVE_PROJECT_PATH = '/content/drive/MyDrive/AI-Video-Clipper-Studio-V3'
LOCAL_PROJECT_PATH = '/content/AI-Video-Clipper-Studio-V3'

# 3. Copiar projeto para ambiente local (mais rápido)
print(f"📦 Copiando projeto de: {DRIVE_PROJECT_PATH}...")

if not os.path.exists(DRIVE_PROJECT_PATH):
    print(f"❌ Erro: Pasta do projeto não encontrada no Drive: {DRIVE_PROJECT_PATH}")
    print("   Certifique-se de ter feito o upload primeiro!")
else:
    # Usar rsync para copiar (rápido e eficiente)
    !rsync -aP --exclude 'venv' --exclude '__pycache__' "{DRIVE_PROJECT_PATH}/" "{LOCAL_PROJECT_PATH}/"
    
    # Entrar na pasta
    os.chdir(LOCAL_PROJECT_PATH)
    print(f"✅ Projeto carregado em: {os.getcwd()}")

## 📦 ETAPA 2: Instalar Dependências

In [ ]:
# Instalar dependências do sistema system (INCLUINDO AUDIO FIX e ESPEAK)
!apt-get update -qq
!apt-get install -y ffmpeg chromium-chromedriver xvfb imagemagick alsa-utils libasound2-dev portaudio19-dev libsndfile1 espeak-ng

# Instalar dependências Python do projeto
print("🐍 Instalando bibliotecas Python...")
!pip install -r colab_requirements.txt

# Hotfix: Corrigir MoviePy SyntaxWarnings (Python 3.12)
print("🔧 Aplicando patch no MoviePy...")
!python fix_moviepy.py

print("✅ Todas as dependências instaladas!")

## 🤖 ETAPA 3: Configurar Ollama (IA Local) e Vosk

In [ ]:
# Instalar dependência essencial (zstd) e Ollama
!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

# Baixar Modelo Vosk (PT-BR) para Transcrição Offline
print("⬇️ Baixando modelo de voz (Vosk PT-BR)...")
!mkdir -p models
!wget -q https://alphacephei.com/vosk/models/vosk-model-small-pt-0.3.zip -O models/model.zip
!unzip -o -q models/model.zip -d models/
!rm models/model.zip
print("✅ Modelo de voz instalado!")

# Iniciar servidor Ollama em background
import subprocess
import time
import os

# Configurar variável de ambiente para o Ollama rodar em background
os.environ['OLLAMA_HOST'] = '127.0.0.1:11434'

print("⏳ Iniciando servidor Ollama...")
subprocess.Popen(['ollama', 'serve'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
time.sleep(5)  # Aguardar inicialização

# Baixar modelo Llama3 (ou outro leve)
print("🧠 Baixando modelo Llama3 (pode demorar um pouco)...")
!ollama pull llama3
print("✅ Ollama pronto para uso!")

## ⚙️ ETAPA 4: Configurar Ambiente (.env)

In [ ]:
# Configurar .env para o Colab
env_content = """
# Ollama
OLLAMA_URL=http://localhost:11434/api/generate
OLLAMA_MODEL=llama3

# Diretórios (Mapeados para o Drive para persistência)
TEMP_DIR=/content/AI-Video-Clipper-Studio-V3/temp
EXPORT_DIR=/content/drive/MyDrive/AI-Video-Clipper-Exports

# Transcrição
WHISPER_LANGUAGE=pt
"""

with open('.env', 'w') as f:
    f.write(env_content)

# Criar diretório de exportação no Drive se não existir
!mkdir -p /content/drive/MyDrive/AI-Video-Clipper-Exports

print("✅ Arquivo .env configurado!")
print("📂 Pasta de exportação: /content/drive/MyDrive/AI-Video-Clipper-Exports")

## 🌐 ETAPA 5: INICIAR INTERFACE WEB

Esta etapa iniciará o servidor e criará um link público para você acessar a interface.

1. Crie uma conta gratuita em [ngrok.com](https://dashboard.ngrok.com/signup)
2. Pegue seu **Authtoken** aqui: [dashboard.ngrok.com/get-started/your-authtoken](https://dashboard.ngrok.com/get-started/your-authtoken)

In [ ]:
import os
from pyngrok import ngrok
import subprocess
import time

# 1. Configurar Ngrok (Token Automático)
print("🔑 Configurando Ngrok...")
token = "2tvNFAWzP9KMYZGpfCqx1EQmmwN_NPCQKjeqHD7pomCtJFVA"
ngrok.set_auth_token(token)
print("✅ Token Ngrok configurado automaticamente!")

# 2. Iniciar Flask App em Background
print("🚀 Iniciando servidor Flask...")
flask_process = subprocess.Popen(['python', 'app.py'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(5) # Aguardar inicialização

# 3. Criar Túnel Público
try:
    # Matar túneis anteriores se houver
    ngrok.kill()
    
    # Criar novo túnel na porta 5005 (Porta do Flask app.py)
    public_url = ngrok.connect(5005)
    print("\n" + "="*50)
    print(f"✅ INTERFACE WEB ONLINE!")
    print(f"👉 Acesse aqui: {public_url}")
    print("="*50 + "\n")
    
    # Manter célula rodando
    print("ℹ️  Mantenha esta célula rodando para usar o site.")
    print("ℹ️  Logs do servidor aparecerão abaixo:")
    
    # Stream logs
    while True:
        line = flask_process.stdout.readline()
        if not line:
            break
        print(line.decode('utf-8').strip())

except Exception as e:
    print(f"❌ Erro ao criar túnel: {e}")
    flask_process.terminate()

## 📤 Salvar Alterações no Drive

In [ ]:
# Se fez alterações no código e quer salvar de volta no Drive:
!rsync -aP --exclude 'venv' --exclude 'temp' --exclude '__pycache__' /content/AI-Video-Clipper-Studio-V3/ /content/drive/MyDrive/AI-Video-Clipper-Studio-V3/
print("✅ Alterações salvas no Drive!")